# Fine-Tuning Models
> Fine-tuning using your own data

In this notebook, we'll use two references:https://huggingface.co/transformers/custom_datasets.html as a guide for our work.  We'll use the HuggingFace dataset we've already created and use it directly!

### Install required packages
Note that this is mostly required if you're on Google Colab.

In [1]:
! pip install transformers
! pip install datasets

     |████████████████████████████████| 3.4 MB 5.0 MB/s 
     |████████████████████████████████| 895 kB 43.4 MB/s 
     |████████████████████████████████| 3.3 MB 43.0 MB/s 
     |████████████████████████████████| 61 kB 302 kB/s 
     |████████████████████████████████| 596 kB 39.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 306 kB 5.2 MB/s 
     |████████████████████████████████| 243 kB 60.9 MB/s 
     |████████████████████████████████| 1.1 MB 55.9 MB/s 
     |████████████████████████████████| 132 kB 73.5 MB/s 
     |████████████████████████████████| 271 kB 75.3 MB/s 
     |████████████████████████████████| 160 kB 72.5 MB/s 
     |████████████████████████████████| 192 kB 68.0 MB/s 


### Import packages of interest

In [2]:
import numpy as np
import pandas as pd

from datasets import load_dataset, load_metric, Dataset
from transformers import pipeline
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

from huggingface_hub import notebook_login

# 0. Log into HuggingFace CLI
Why are we doing this? Below, we'll use our own user accounts to grab datasets and upload models. If we don't do this, we'll have to pass in the auth token over. This isn't bad, but let's streamline our efforts!

In [3]:
!git config --global credential.helper store

In [4]:
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


# 1. Load data from HuggingFace Hub or from disk

In [5]:
ds_path = 'charreaubell/demo_data'
demo_ds = load_dataset(ds_path)

Downloading:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

Using custom data configuration charreaubell--demo_data-8b5785817027195e


  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/7.10k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.91k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/7.64k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/charreaubell--demo_data-8b5785817027195e/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

# 2. Pre-process inputs
What's a tokenizer and what does it do? Let's learn more using Huggingface's [instruction on tokenizers](https://huggingface.co/course/chapter2/4?fw=pt). Then, let's try it on our own!

In [6]:
#instantiate tokenizer
tokenier = AutoTokenizer.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [7]:
#define tokenizing function
def tokenize_inputs(example):
    return tokenier(example['text'], truncation= True)

In [8]:
#do the tokenizing using map function
tokenized_ds = demo_ds.map(tokenize_inputs, batched=True,
                           remove_columns = ['age', 'article_id', 'college_major',
                                             'first_name', 'last_name', 'years_of_journalism',
                                             'text'])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [9]:
tokenized_ds

DatasetDict({
    valid: Dataset({
        features: ['attention_mask', 'input_ids', 'label'],
        num_rows: 4
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label'],
        num_rows: 4
    })
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label'],
        num_rows: 12
    })
})

## An aside on tokenizer functionality
We can do many things with tokenizers to help us to tokenize our data and process it. Let's check out these outputs further.

In [12]:
#check out input IDs
print(tokenized_ds['train']['input_ids'][0])

#compare against the text
demo_ds['train']['text'][0]

[101, 1000, 4863, 2000, 2033, 2153, 2339, 1045, 5807, 2102, 21910, 1029, 1000, 2002, 2356, 1012, 1000, 2035, 1996, 2500, 2079, 1998, 6343, 2412, 4152, 14248, 2005, 2725, 2061, 1012, 1045, 2323, 2175, 2055, 2108, 3407, 3974, 2000, 21910, 2545, 2138, 1045, 2113, 2008, 1045, 2123, 2102, 1029, 2008, 2015, 2054, 2115, 2063, 4129, 2033, 1029, 102]


'"Explain to me again why I shouldnt cheat?" he asked. "All the others do and nobody ever gets punished for doing so. I should go about being happy losing to cheaters because I know that I dont? Thats what youre telling me?'

In [14]:
#check out the length of the list of lists
print(len(tokenized_ds['train']['input_ids']))

#check out the length of a single element
len(tokenized_ds['train']['input_ids'][0])

12


57

In [16]:
#convert input_ids to token representation
input0_tokens = tokenier.convert_ids_to_tokens(tokenized_ds['train']['input_ids'][0])
input0_tokens

['[CLS]',
 '"',
 'explain',
 'to',
 'me',
 'again',
 'why',
 'i',
 'shouldn',
 '##t',
 'cheat',
 '?',
 '"',
 'he',
 'asked',
 '.',
 '"',
 'all',
 'the',
 'others',
 'do',
 'and',
 'nobody',
 'ever',
 'gets',
 'punished',
 'for',
 'doing',
 'so',
 '.',
 'i',
 'should',
 'go',
 'about',
 'being',
 'happy',
 'losing',
 'to',
 'cheat',
 '##ers',
 'because',
 'i',
 'know',
 'that',
 'i',
 'don',
 '##t',
 '?',
 'that',
 '##s',
 'what',
 'your',
 '##e',
 'telling',
 'me',
 '?',
 '[SEP]']

In [19]:
#see what this looks like as a string
print(tokenier.convert_tokens_to_string(input0_tokens))

#another method directly from the input ids
tokenier.decode(tokenized_ds['train']['input_ids'][0])

[CLS] " explain to me again why i shouldnt cheat? " he asked. " all the others do and nobody ever gets punished for doing so. i should go about being happy losing to cheaters because i know that i dont? thats what youre telling me? [SEP]


'[CLS] " explain to me again why i shouldnt cheat? " he asked. " all the others do and nobody ever gets punished for doing so. i should go about being happy losing to cheaters because i know that i dont? thats what youre telling me? [SEP]'

In [21]:
#other information about tokenizer
tokenier.vocab_size

#see actual tokenizer vocab (we've abbreviated here)
tokenier.vocab

30522

## An aside on dynamically padded batch size
HF has the capacity to dynamically pad your batches such that each input is only as long as any given input in the batch. This helps with memory.You can learn more [here](https://huggingface.co/course/chapter3/2?fw=pt). For now, we'll simply instantiate a data collator and use it during training to demonstrate how we can do this.

In [23]:
#Instantiate data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenier)

# 3. Train model

In [24]:
#recall that the dataset carries information about the classes
demo_ds['train'].features['label']

ClassLabel(num_classes=5, names=['engineering', 'humanities', 'prelaw', 'premed', 'science'], names_file=None, id=None)

In [25]:
#get the number of classes
no_classes = demo_ds['train'].features['label'].num_classes
no_classes

5

In [26]:
#get label conversions
id2label = {ind:label for ind, label in enumerate(demo_ds['train'].features['label'].names)}
label2id = {label:ind for ind, label in id2label.items()}

In [ ]:
#check it out


## Define model and task architecture

In [28]:
# Choose the model type and instantiate it for the task
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=no_classes, id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier

## Define settings for basic model training and train

In [29]:
#set training arguments
training_args = TrainingArguments('test-trainer', logging_strategy='epoch')

#setup training loop with arguments
trainer= Trainer(model=model, args=training_args, data_collator=data_collator, tokenizer=tokenier, train_dataset=tokenized_ds['train'])

#train
trainer.train()

***** Running training *****
  Num examples = 12
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6


Step,Training Loss
2,1.606400
4,1.511600
6,1.460100




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=6, training_loss=1.5260037581125896, metrics={'train_runtime': 22.2793, 'train_samples_per_second': 1.616, 'train_steps_per_second': 0.269, 'total_flos': 924216529560.0, 'train_loss': 1.5260037581125896, 'epoch': 3.0})

### Reflect and Discuss
* Practically speaking, how is the model performing?

## Training with performance metrics and saving checkpoints of the model

In [30]:
#load a metric
metric = load_metric('accuracy')

#define the metric behavior
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

In [36]:
#set new training arguments
training_args = TrainingArguments("test-trainer",
                                  logging_strategy = "epoch",
                                  evaluation_strategy = "epoch",
                                  save_strategy = "epoch",
                                  report_to = 'all'
                                 )

#setup training loop
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenier,
    data_collator=data_collator,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['valid'],
    compute_metrics = compute_metrics
)

#train model
trainer.train()

PyTorch: setting up devices
***** Running training *****
  Num examples = 12
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6


Epoch,Training Loss,Validation Loss,Accuracy
1,1.347100,1.560933,0.500000
2,1.296900,1.532672,0.500000
3,1.174600,1.517963,0.500000


***** Running Evaluation *****
  Num examples = 4
  Batch size = 8
Saving model checkpoint to test-trainer/checkpoint-2
Configuration saved in test-trainer/checkpoint-2/config.json
Model weights saved in test-trainer/checkpoint-2/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-2/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-2/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4
  Batch size = 8
Saving model checkpoint to test-trainer/checkpoint-4
Configuration saved in test-trainer/checkpoint-4/config.json
Model weights saved in test-trainer/checkpoint-4/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-4/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-4/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4
  Batch size = 8
Saving model checkpoint to test-trainer/checkpoint-6
Configuration saved in test-trainer/checkpoint-6/config.json
Model w

TrainOutput(global_step=6, training_loss=1.2728726069132488, metrics={'train_runtime': 33.6518, 'train_samples_per_second': 1.07, 'train_steps_per_second': 0.178, 'total_flos': 924216529560.0, 'train_loss': 1.2728726069132488, 'epoch': 3.0})

### Reflect and Discuss
* What new observations are present during model training?
* What comments can you make on the performance of the model now?
* What metrics are appropriate for your application?
* Consider that model training is done in-memory (the model weights are updated in memory, but not returned), and both of our `Trainer`s trained our model `model`. After basic training from Step 9 and training from Step 10, how many epochs has the model been trained?

## A brief aside on resuming training from checkpoints

In [37]:
#update the number of epochs (or steps) that you want to train for
trainer.args.num_train_epochs = 6

In [38]:
#train some more, resuming from checkpoint
trainer.train(resume_from_checkpoint=True)

Loading model from test-trainer/checkpoint-6).
***** Running training *****
  Num examples = 12
  Num Epochs = 6
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 12
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 3
  Continuing training from global step 6
  Will skip the first 3 epochs then the first 0 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


0it [00:00, ?it/s]

Epoch,Training Loss,Validation Loss,Accuracy
4,1.088400,1.517963,0.500000
5,1.192200,1.517963,0.500000
6,1.135500,1.517963,0.500000


***** Running Evaluation *****
  Num examples = 4
  Batch size = 8
Saving model checkpoint to test-trainer/checkpoint-8
Configuration saved in test-trainer/checkpoint-8/config.json
Model weights saved in test-trainer/checkpoint-8/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-8/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-8/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4
  Batch size = 8
Saving model checkpoint to test-trainer/checkpoint-10
Configuration saved in test-trainer/checkpoint-10/config.json
Model weights saved in test-trainer/checkpoint-10/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-10/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-10/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4
  Batch size = 8
Saving model checkpoint to test-trainer/checkpoint-12
Configuration saved in test-trainer/checkpoint-12/config.json


TrainOutput(global_step=12, training_loss=0.5693502227465311, metrics={'train_runtime': 30.8839, 'train_samples_per_second': 2.331, 'train_steps_per_second': 0.389, 'total_flos': 1870167154440.0, 'train_loss': 0.5693502227465311, 'epoch': 6.0})

## A brief aside on performance metrics
You may want to use other performance metrics than accuracy. Here are some [metrics available through Huggingface](https://huggingface.co/metrics). If you check out the metrics folder on the [Huggingface datasets](https://github.com/huggingface/datasets) repository, you'll be able to see what's necessary if you need to define another metric. Let's try a different metric!

In [39]:
from sklearn.metrics import precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    
    #get predictions by using index of max logit
    predictions = np.argmax(logits, axis=-1)
    
    #calculate classification report
    perfs = precision_recall_fscore_support(labels, predictions, average='macro', zero_division=0)
    perf_dict = dict(zip(['precision', 'recall', 'fscore'], perfs[:3]))
    
    #return dictionary
    return perf_dict

In [41]:
#setup training loop
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenier,
    data_collator=data_collator,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['valid'],
    compute_metrics=compute_metrics
)

trainer.train()

***** Running training *****
  Num examples = 12
  Num Epochs = 6
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 12


Epoch,Training Loss,Validation Loss,Precision,Recall,Fscore
1,0.984100,1.497798,0.166667,0.333333,0.222222
2,1.005300,1.488669,0.166667,0.333333,0.222222
3,0.853700,1.462136,0.166667,0.333333,0.222222
4,0.721300,1.429380,0.166667,0.333333,0.222222
5,0.700400,1.408053,0.166667,0.333333,0.222222
6,0.684600,1.401328,0.166667,0.333333,0.222222


***** Running Evaluation *****
  Num examples = 4
  Batch size = 8
Saving model checkpoint to test-trainer/checkpoint-2
Configuration saved in test-trainer/checkpoint-2/config.json
Model weights saved in test-trainer/checkpoint-2/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-2/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-2/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4
  Batch size = 8
Saving model checkpoint to test-trainer/checkpoint-4
Configuration saved in test-trainer/checkpoint-4/config.json
Model weights saved in test-trainer/checkpoint-4/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-4/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-4/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4
  Batch size = 8
Saving model checkpoint to test-trainer/checkpoint-6
Configuration saved in test-trainer/checkpoint-6/config.json
Model w

TrainOutput(global_step=12, training_loss=0.8248956898848215, metrics={'train_runtime': 72.7938, 'train_samples_per_second': 0.989, 'train_steps_per_second': 0.165, 'total_flos': 1836013576080.0, 'train_loss': 0.8248956898848215, 'epoch': 6.0})

## A brief aside on model training - TRY IT YOURSELF!
One of several points of ambiguity when training models is how long should they train for? A way to approach this is to monitor the models and run them repeatedly, starting from the last checkpoint. Another way is through training a number of epochs (if you model trains quickly enough) and then always load the best model according to some metric at the end. Let's take a look at this.

We can realize this through `TrainingArguments`! In your breakout rooms, add the parameters which will enable the following:
1. Load the best model at the end
2. Set the metric for using the best model to one of our evaluation metrics
3. Examine the `greater_is_better` parameter. Do you need to change it?
4. Change the number of training epochs to something larger.
5. Decrease the training batch size.
6. Decrease the eval batch size.
7. How can you change the logging, evaluation, and save strategies to step? What else might you need to change depending on the interval of steps that you want these activities to occur?

Make sure this works, so run the cell!

In [44]:
#set new training arguments
training_args = TrainingArguments("test-trainer",
                                  logging_strategy = "steps",
                                  evaluation_strategy="steps",
                                  save_strategy='steps',
                                  num_train_epochs = 10,
                                  per_device_train_batch_size = 32,
                                  per_device_eval_batch_size = 32,
                                  greater_is_better = True,
                                  load_best_model_at_end = True
                                  )

#setup training loop
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenier,
    data_collator=data_collator,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['valid'],
    compute_metrics=compute_metrics
)

#train model
trainer.train()

using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 12
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 10


Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=10, training_loss=0.41094250679016114, metrics={'train_runtime': 70.0446, 'train_samples_per_second': 1.713, 'train_steps_per_second': 0.143, 'total_flos': 3322211713200.0, 'train_loss': 0.41094250679016114, 'epoch': 10.0})

# 4. Using trained model with `Trainer`
## Evaluate

## Predict

# 5. Sharing and saving your model
## Using `Trainer`
During training and using the Trainer class, you can also upload your model directly to HuggingFace Hub as it trains. Read more about this process on the [HF course documentation](https://huggingface.co/course/chapter4/3?fw=pt).

Let's check out how to do this. It's as simple as modifying our `TrainingArguments`! Don't forget to have already logged in using your authorization token or use the `use_auth_token` paramter to access your HF account. You'll need to have git-lfs installed to use this feature, so if you're on Google Colab, you can execute the line below. You can also `conda install -c conda-forge git-lfs` if you're using a conda environment.

In [ ]:
#!apt-get install git-lfs

In [ ]:
#set new training arguments
training_args = TrainingArguments("test-trainer",
                                  overwrite_output_dir=True,
                                  logging_strategy = "epoch",
                                  evaluation_strategy="epoch",
                                  save_strategy='epoch',
                                  #new arguments
                                  report_to='all')

#setup training loop
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['valid'],
    compute_metrics=compute_metrics
)

#train model
trainer.train()

#### Reflect
Visit your repository and take a look to make sure your model uploaded. Answer the following questions:
* Where did your model save locally (directory)?
* What are the contents of the saved model?
* Investigate your uploaded model.

In [ ]:
#it's recommended to push the final version to HF after training completes.


#### Reflect
Visit your repository once more (you'll likely need to refresh) and check out the changes.
* What is different from the uploads during training?
* What do you observe about the model cards?

## Fine-grained save/push access
You can also push the model and/or tokenizer directly using the `push_to_hub` methods in their classes. You can learn more about this [in the Huggingface docs.](https://huggingface.co/course/chapter4/3?fw=pt) An example of using trainer to save your entire model locally is shown below.

In [ ]:
trainer.save_model('test-trainer')

# 6. Using your fine-tuned model

In [ ]:
#create pipeline from your classifier


#optionally, load from HF
#mag_classifier = pipeline('text-classification', model='charreaubell/distilbert-magazine-classifier', use_auth_token=True)

#get output


In [ ]:
#do inference using trained model


## Reflect and discuss: Breakout Rooms
You've successfully trained a model - great job!! Now, let's focus on what YOU need to do for your task. Using the [Transformer Notebooks](https://huggingface.co/docs/transformers/notebooks) and use the `Open in Colab` badge, explore what this task looks like. Note that even if your modality is different, you may be able to directly still use these notebooks with a few changes!